---


## CARGA DE LIBRERÍA Y DATOS

---

In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from scipy.stats import rankdata
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chi2, f_oneway,chi2_contingency
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.utils import shuffle

In [104]:
df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
test = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/sample_submission.csv")

---

## DEFINICIÓN PREPROCESADO

---

In [105]:
cols_to_int = ['age', 'sex', 'cp', 'restecg']

rename_dict = {
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal"
}

cols_a_clippear = [
    'tension_en_descanso', 'colesterol',
    'latidos_por_minuto', 'cambio_linea_corazon_ejercicio'
]

categorical_cols_to_round = [
    'num_venas_grandes', 'estado_corazon_thal', 'sexo',
    'tipo_dolor_pecho', 'dolor_pecho_con_ejercicio',
    'azucar', 'forma_linea_corazon_ejercicio', 'electro_en_descanso'
]

---

FUNCIONES DE PREPROCESADO

---

In [106]:
def limpieza_inicial(df):
    """
    Realiza conversiones de tipos, renombres y limpieza básica de errores (-9).
    Se puede aplicar a todo el dataset antes del split.
    """
    df = df.copy()

    for col in cols_to_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    object_cols = df.select_dtypes(include=['object']).columns
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.rename(columns=rename_dict)
    df.replace([-9, -9.0], np.nan, inplace=True)

    return df

In [107]:
def limpiar_ceros_fisiologicos(X):
    X = X.copy()
    cols_imposibles_con_cero = ['tension_en_descanso', 'colesterol']
    for col in cols_imposibles_con_cero:
        if col in X.columns:
            X[col] = X[col].replace({0: np.nan, 0.0: np.nan})
    return X

In [108]:
def clipear_outliers(X):
    X = X.copy()
    for col in cols_a_clippear:
        if col in X.columns:
            p1 = X[col].quantile(0.01)
            p99 = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=p1, upper=p99)
    return X

In [109]:
def crear_flags_mnar(df):
    """
    ACTUALIZADO (Estrategia Pruning):
    Solo creamos flag para 'num_venas_grandes'.
    'estado_corazon_thal_is_missing' se considera ruido (Grupo 1) y no se genera.
    """
    df_new = df.copy()
    # Solo venas, thal is missing se elimina por V=0.040
    cols_mnar = ['num_venas_grandes']
    for col in cols_mnar:
        if col in df_new.columns:
            df_new[f'{col}_is_missing'] = df_new[col].isna().astype(int)
    return df_new

In [110]:
class RobustKNNImputerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.scaler = RobustScaler()
        self.imputer = KNNImputer(n_neighbors=n_neighbors, weights='distance')
        self.feature_names_in_ = None

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns if hasattr(X, 'columns') else [f"feat_{i}" for i in range(X.shape[1])]
        X_scaled = self.scaler.fit_transform(X)
        self.imputer.fit(X_scaled)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        X_imputed_scaled = self.imputer.transform(X_scaled)
        X_imputed = self.scaler.inverse_transform(X_imputed_scaled)
        return pd.DataFrame(X_imputed, columns=self.feature_names_in_, index=X.index)

    def set_output(self, *, transform=None):
        return self

In [111]:
def redondear_imputaciones(X):
    X = X.copy()
    for col in categorical_cols_to_round:
        if col in X.columns:
            X[col] = X[col].round()
    return X

In [112]:
def aplicar_feature_engineering_avanzado(df):
    """
    ACTUALIZADO:
    1. Se eliminan cálculos innecesarios (rango_colesterol, porcentaje_freq).
    2. Se calculan las variables compuestas necesarias antes de borrar las fuentes.
    """
    df = df.copy()

    # 1. Flag Depresión ST (Usada en Score Stress)
    if 'cambio_linea_corazon_ejercicio' in df.columns:
        df['flag_depresion_st'] = (df['cambio_linea_corazon_ejercicio'] > 0).astype(int)

    # 2. Flag Hipertensión (Absorbe info de tension_en_descanso)
    if 'tension_en_descanso' in df.columns:
        df['flag_hipertension'] = (df['tension_en_descanso'] > 130).astype(int)

    # NOTA: Eliminado calculo de 'porcentaje_frecuencia_max' (Grupo 2 - Redundancia negativa)
    # NOTA: Eliminado calculo de 'rango_colesterol' (Grupo 1 - Ruido)

    # 3. Score Respuesta al Estrés (Driver Tier 1)
    if 'dolor_pecho_con_ejercicio' in df.columns and 'flag_depresion_st' in df.columns:
        df['score_respuesta_stress'] = df['dolor_pecho_con_ejercicio'] + df['flag_depresion_st']

    # 4. Carga de Comorbilidad (Absorbe azucar y electro)
    cols_comorbilidad = ['azucar', 'flag_hipertension', 'electro_en_descanso']
    if set(cols_comorbilidad).issubset(df.columns):
        # Nos aseguramos que electro sea binario para la suma (0 = normal, 1,2 = anormal)
        electro_punto = (df['electro_en_descanso'] > 0).astype(int)
        df['carga_comorbilidad'] = df['azucar'] + df['flag_hipertension'] + electro_punto

    return df

In [113]:
def ejecutar_pruning_agresivo(df):
    """
    Elimina las variables de Grupo 1 (Ruido) y Grupo 2 (Redundancia/Absorbidas)
    después de haberlas utilizado para crear las variables compuestas.
    """
    df = df.copy()

    vars_a_eliminar = [
        # GRUPO 1: Ruido Puro
        'electro_en_descanso', # Usado en carga, adios.
        'colesterol',          # Inutil en este dataset, adios.
        # 'rango_colesterol' y 'thal_missing' ya no se generan.

        # GRUPO 2: Redundantes / Absorbidas
        'azucar',               # Absorbida en carga, adios.
        'tension_en_descanso'   # Absorbida en flag_hipertension, adios.
    ]

    # Eliminamos solo si existen (para evitar errores)
    cols_existentes = [c for c in vars_a_eliminar if c in df.columns]
    if cols_existentes:
        df = df.drop(columns=cols_existentes)

    return df

In [114]:
def optimizar_k_knn(X_train, k_range=[3, 5, 7, 9, 11, 15]):
    # Versión simplificada de la original
    X_temp = limpiar_ceros_fisiologicos(X_train)
    X_temp = clipear_outliers(X_temp)
    X_complete = X_temp.dropna().copy()
    if len(X_complete) < 50: return 5

    rmse_scores = {}
    scaler = RobustScaler()
    X_scaled_array = scaler.fit_transform(X_complete)
    np.random.seed(42)
    mask = np.random.rand(*X_scaled_array.shape) < 0.1
    X_missing_sim = X_scaled_array.copy()
    X_missing_sim[mask] = np.nan

    for k in k_range:
        imputer = KNNImputer(n_neighbors=k, weights='distance')
        X_imputed = imputer.fit_transform(X_missing_sim)
        error = np.sqrt(mean_squared_error(X_scaled_array[mask], X_imputed[mask]))
        rmse_scores[k] = error

    return min(rmse_scores, key=rmse_scores.get)

---
## APLICACIÓN DEL FLUJO DE PREPROCESADO DEFINIDO

---

In [115]:
df_train = df.copy()
df_test = test.copy()

df_train = limpieza_inicial(df_train)
df_test = limpieza_inicial(df_test)

target = "label"

X_train = df_train.drop(columns=target)
y_train = df_train[target]

if target in df_test.columns:
    X_test = df_test.drop(columns=target)
    y_test = df_test[target]
else:
    X_test = df_test.copy()

best_k = optimizar_k_knn(X_train)

# 3. Definición del Pipeline
pipeline_feature_engineering = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear_outliers', FunctionTransformer(clipear_outliers, validate=False)),
    ('mnar_flags', FunctionTransformer(crear_flags_mnar, validate=False)),
    ('imputacion_robusta', RobustKNNImputerWrapper(n_neighbors=best_k)),
    ('rounding', FunctionTransformer(redondear_imputaciones, validate=False)),
    ('feature_engineering', FunctionTransformer(aplicar_feature_engineering_avanzado, validate=False)),
    # CAMBIO CLAVE: COMENTAMOS ESTA LÍNEA (No borramos columnas en segundo intento del stacking)
    ('pruning_agresivo', FunctionTransformer(ejecutar_pruning_agresivo, validate=False)),
    # ---------------------------------------------------------
    ('final_scaler', RobustScaler())
]).set_output(transform="pandas")

---
## EJECUCIÓN Y VERIFICACIÓN DE TRANSFORMACIÓN

---

In [116]:
print("Ajustando pipeline con estrategia de Pruning Agresivo...")
X_train_prep = pipeline_feature_engineering.fit_transform(X_train)
y_train_prep = y_train.copy()

X_test_prep = pipeline_feature_engineering.transform(X_test)
print("\n--- Proceso finalizado ---")
print(f"Dimensiones Train final: {X_train_prep.shape}")
print(f"Columnas finales en el dataset: \n{X_train_prep.columns.tolist()}")

Ajustando pipeline con estrategia de Pruning Agresivo...

--- Proceso finalizado ---
Dimensiones Train final: (732, 14)
Columnas finales en el dataset: 
['edad', 'sexo', 'tipo_dolor_pecho', 'latidos_por_minuto', 'dolor_pecho_con_ejercicio', 'cambio_linea_corazon_ejercicio', 'forma_linea_corazon_ejercicio', 'num_venas_grandes', 'estado_corazon_thal', 'num_venas_grandes_is_missing', 'flag_depresion_st', 'flag_hipertension', 'score_respuesta_stress', 'carga_comorbilidad']


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE ENSAMBLES

---

---
### DEFINICIÓN MODELOS PREVIO AL STACKING

---

In [ ]:
# 1. Logistic Regression
# Params: {'lr__C': 1, 'lr__class_weight': None, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}
best_lr = LogisticRegression(
    C=1,
    class_weight=None,
    penalty='l2',
    solver='lbfgs',
    max_iter=1000, # Aumentado por seguridad de convergencia
    random_state=42)

# 2. Random Forest
# Params: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
best_rf = RandomForestClassifier(
    class_weight='balanced',
    max_depth=5,
    min_samples_split=5,
    n_estimators=100,
    random_state=42)

# 3. XGBoost
# Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
best_xgb = XGBClassifier(
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
    subsample=1.0,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42)

# 4. Balanced Random Forest
# Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
best_brf = BalancedRandomForestClassifier(
    max_depth=None,
    min_samples_split=2,
    n_estimators=200,
    random_state=42)

# 5. Easy Ensemble
# Params: {'n_estimators': 10, 'sampling_strategy': 'auto'}
best_eec = EasyEnsembleClassifier(
    n_estimators=10,
    sampling_strategy='auto',
    random_state=42)

---
ENTRENAMIENTO Y PREDICCIONES SOBRE TEST SET

---

In [ ]:
#0.5489
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', best_lr),
        ('rf', best_rf),
        ('xgb', best_xgb),
        ('brf', best_brf),
        ('eec', best_eec)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5, # Cross-validation interno para las meta-features
    n_jobs=-1)

# Entrenamos con los datos PREPROCESADOS
stacking_clf.fit(X_train_prep, y_train)

# Usamos el modelo ya entrenado arriba
y_test_pred = stacking_clf.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_stacking_pruned_opt1.csv", index=False)
    print("¡Archivo 'submission_stacking_pruned_opt1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_stacking_pruned_opt1.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      3
4   4      0


---
ENTRENAMIENTO Y PREDICCIONES SOBRE TEST SET SIN PRUNING (sin borrar columnas)

---

In [ ]:
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', best_lr),
        ('rf', best_rf),
        ('xgb', best_xgb),
        ('brf', best_brf),
        ('eec', best_eec)
    ],
    # Cambio: Añadimos class_weight='balanced' al meta-modelo también
    final_estimator=LogisticRegression(max_iter=1000, class_weight='balanced'),
    cv=5,
    n_jobs=-1)

# Entrenamos con los datos PREPROCESADOS
stacking_clf.fit(X_train_prep, y_train)

# Usamos el modelo ya entrenado arriba
y_test_pred = stacking_clf.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_stacking_opt1.csv", index=False)
    print("¡Archivo 'submission_stacking_opt1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_stacking_opt1.csv' guardado con éxito!
   ID  label
0   0      4
1   1      0
2   2      0
3   3      3
4   4      0


---
ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST SET CON SVM Y XGBOOST COMO META-MODELO

In [ ]:
best_svm = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    class_weight=None,
    probability=True,
    random_state=42)

# 2. DEFINIMOS EL META-MODELO
meta_learner = XGBClassifier(
    n_estimators=100,
    max_depth=3,          # Poca profundidad para evitar memorizar
    learning_rate=0.05,   # Aprendizaje suave
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

# 3. EL STACKING DEFINITIVO (Árboles + SVM)
print("🚀 Entrenando Stacking Classifier con SVM...")

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', best_lr),
        ('rf', best_rf),
        ('xgb', best_xgb),
        ('brf', best_brf),
        ('eec', best_eec),
        ('svm', best_svm)
    ],
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1,
    passthrough=False
)

# Entrenamos con los datos PRUNED (tu mejor preprocesado)
stacking_clf.fit(X_train_prep, y_train)
print("✅ Entrenamiento finalizado.")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_stacking_pruned_opt2.csv", index=False)
    print("¡Archivo 'submission_stacking_pruned_opt2.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

🚀 Entrenando Stacking Classifier con SVM...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:45:47] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Entrenamiento finalizado.
¡Archivo 'submission_stacking_pruned_opt2.csv' guardado con éxito!
   ID  label
0   0      4
1   1      0
2   2      0
3   3      3
4   4      0


---

### DEFINICIÓN MODELOS PREVIO AL VOTING

---

In [ ]:
#!pip install catboost

In [ ]:
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("⚠️ CatBoost no instalado. Usaremos XGBoost en su lugar.")

best_svm = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    probability=True, # OBLIGATORIO para Soft Voting
    random_state=42
)


best_lr = LogisticRegression(
    C=0.1,
    solver='lbfgs',
    max_iter=1000,
    random_state=42
)


best_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    class_weight='balanced',
    random_state=42
)

# El Especialista en Boosting (CatBoost o tu XGBoost anterior)
if CATBOOST_AVAILABLE:
    print("🐱 Configurando CatBoost...")
    # CatBoost suele funcionar bien con parámetros por defecto
    best_boost = CatBoostClassifier(
        iterations=500,
        depth=4,
        learning_rate=0.05,
        loss_function='MultiClass',
        verbose=0,
        random_seed=42
    )
else:
    # Tu XGBoost anterior
    best_boost = best_xgb

🐱 Configurando CatBoost...


---
ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST SET

---

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('svm', best_svm),
        ('lr', best_lr),
        ('rf', best_rf),
        ('boost', best_boost)
    ],
    voting='soft', # Usa probabilidades
    weights=[4, 1, 1, 1],
    n_jobs=-1
)


voting_clf.fit(X_train_prep, y_train)

y_test_pred = voting_clf.predict(X_test_prep)

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_voting_pruned_opt1.csv", index=False)
    print("¡Archivo 'submission_voting_pruned_opt1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

¡Archivo 'submission_voting_pruned_opt1.csv' guardado con éxito!
   ID  label
0   0      2
1   1      0
2   2      0
3   3      2
4   4      0


---
VOTING SIN BOOSTING Y RF

---

In [ ]:
best_svm = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    probability=True,  # Vital para mezclar con Soft Voting
    random_state=42)

best_lr = LogisticRegression(
    C=0.08,
    solver='lbfgs',
    class_weight=None,
    random_state=42,
    max_iter=1000)

In [ ]:
voting_clf = VotingClassifier(
    estimators=[
        ('svm', best_svm),
        ('lr', best_lr)
    ],
    voting='soft',         # Suma de probabilidades
    weights=[4, 1],        # <--- CLAVE: SVM vale 4 veces más. LR solo desempata.
    n_jobs=-1
)

voting_clf.fit(X_train_prep, y_train)

y_test_pred = voting_clf.predict(X_test_prep)

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_voting_pruned_opt_2.csv", index=False)
    print("¡Archivo 'submission_logreg_pruned_opt_2.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

¡Archivo 'submission_logreg_pruned_opt_2.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


---
## PSEUDO-LABELLING SVM

---

In [ ]:
# MEJOR MODELO (SVM 0.565)
best_svm = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    probability=True,
    random_state=42)

# 2. ENTRENAMIENTO INICIAL (Ronda 1)
print("Ronda 1: Entrenando SVM con datos originales...")
best_svm.fit(X_train_prep, y_train)

# 3. GENERAR PSEUDO-LABELS
print("Generando predicciones de confianza sobre el Test...")
y_test_proba = best_svm.predict_proba(X_test_prep)
y_test_pred = best_svm.predict(X_test_prep)

# Definimos un umbral de confianza alto (ej. 70% por problema dífic
# Solo confiamos en el modelo si está muy seguro.
CONFIDENCE_THRESHOLD = 0.70

# Identificamos los índices del test donde la confianza supera el umbral
# np.max(y_test_proba, axis=1) nos da la probabilidad de la clase ganadora
confident_indices = np.where(np.max(y_test_proba, axis=1) > CONFIDENCE_THRESHOLD)[0]

print(f"Se han encontrado {len(confident_indices)} muestras 'seguras' en el Test para añadir al Train.")

Ronda 1: Entrenando SVM con datos originales...
Generando predicciones de confianza sobre el Test...
Se han encontrado 61 muestras 'seguras' en el Test para añadir al Train.


In [ ]:
# 4. CREAR NUEVO DATASET DE ENTRENAMIENTO (Train + Pseudo-Test)
if len(confident_indices) > 0:
    # Extraemos las features y las predicciones de esas filas seguras
    X_pseudo = X_test_prep.iloc[confident_indices]
    y_pseudo = y_test_pred[confident_indices]

    # Convertimos y_pseudo a Series compatible con y_train
    y_pseudo = pd.Series(y_pseudo, index=X_pseudo.index)

    # Concatenamos
    X_train_augmented = pd.concat([X_train_prep, X_pseudo])
    y_train_augmented = pd.concat([y_train, y_pseudo])

    # Barajamos para que el modelo no aprenda orden
    X_train_augmented, y_train_augmented = shuffle(X_train_augmented, y_train_augmented, random_state=42)

    print(f"Nuevo tamaño de Train: {len(X_train_augmented)} (Original: {len(X_train_prep)})")

    # 5. RE-ENTRENAMIENTO
    print(" Ronda 2: Re-entrenando SVM con datos aumentados (Pseudo-Labeling)...")
    best_svm.fit(X_train_augmented, y_train_augmented)
else:
    print("No hubo suficientes predicciones seguras. Se mantiene el modelo original.")

print("Generando predicciones finales...")
y_final_pred = best_svm.predict(X_test_prep)

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_svm_labelling_pruned_opt_1.csv", index=False)
    print("¡Archivo 'submission_svm_labelling_pruned_opt_1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

Nuevo tamaño de Train: 793 (Original: 732)
 Ronda 2: Re-entrenando SVM con datos aumentados (Pseudo-Labeling)...
Generando predicciones finales...
¡Archivo 'submission_svm_labelling_pruned_opt_1.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


---

## SVM OPTIMIZADO Y PRUNED + STACKING FULL DATA

---

In [ ]:
#1. SVM con mejor rendimiento bajo preprocesado avanzado e inginería de características
# X_train_prep y X_test_prep ya existen.
best_svm = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    class_weight=None,
    probability=True,
    random_state=42
)

# Entrenamos con los datos limpios y podados
best_svm.fit(X_train_prep, y_train)

# Obtenemos PROBABILIDADES (no etiquetas)
probs_svm = best_svm.predict_proba(X_test_prep)
print("Probabilidades SVM calculadas.")

Probabilidades SVM calculadas.


In [ ]:
# 2. Preparamos los datos "sucios" (completos pero no rotos) para el Stacking
# Usamos df (train) y df_test (test) originales, solo imputando nulos básicos
# sin borrar columnas ni ingeniería compleja.

# Limpieza Train
df_clean = limpieza_inicial(df)
X_full_train = df_clean.drop('label', axis=1) if 'label' in df_clean.columns else df_clean
y_full_train = df_clean['label'] if 'label' in df_clean.columns else y_train

# Limpieza Test
df_test_clean_initial = limpieza_inicial(df_test)
X_full_test = df_test_clean_initial.copy()
if 'label' in X_full_test.columns:
    X_full_test = X_full_test.drop('label', axis=1)

# Aplicamos correcciones fisiológicas y outliers (mínimo)
X_full_train = limpiar_ceros_fisiologicos(X_full_train)
X_full_train = clipear_outliers(X_full_train)

X_full_test = limpiar_ceros_fisiologicos(X_full_test)
X_full_test = clipear_outliers(X_full_test)

# Imputación simple (Mediana) para rellenar los NaNs generados
imputer_stacking = SimpleImputer(strategy='median')
X_full_train = pd.DataFrame(imputer_stacking.fit_transform(X_full_train), columns=X_full_train.columns)
X_full_test = pd.DataFrame(imputer_stacking.transform(X_full_test), columns=X_full_test.columns)

In [ ]:
#Definición stacking del 0.57

lr_base = LogisticRegression(C=1, penalty='l2', solver='lbfgs', max_iter=1000, random_state=42)
rf_base = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=5, class_weight='balanced', random_state=42)
xgb_base = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, subsample=1.0, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
brf_base = BalancedRandomForestClassifier(n_estimators=200, min_samples_split=2, random_state=42)
eec_base = EasyEnsembleClassifier(n_estimators=10, sampling_strategy='auto', random_state=42)

# 2.3 Construimos el Stacking
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', lr_base),
        ('rf', rf_base),
        ('xgb', xgb_base),
        ('brf', brf_base),
        ('eec', eec_base)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5,
    n_jobs=-1
)

# Entrenamos con los datos COMPLETOS pero SANEADOS
stacking_clf.fit(X_full_train, y_full_train)

# Obtenemos PROBABILIDADES
probs_stacking = stacking_clf.predict_proba(X_full_test)
print("Probabilidades Stacking calculadas.")

Probabilidades Stacking calculadas.


In [ ]:
# Realizando Blending Híbrido

W_SVM = 0.5      # Peso del SVM (imputación avanzada, inginiería de características y variables pruned)
W_STACKING = 0.5 # Peso del Stacking (datos completos y "sucios")

print(f"   Pesos -> SVM: {W_SVM} | Stacking: {W_STACKING}")

probs_final = (probs_svm * W_SVM) + (probs_stacking * W_STACKING)
y_pred_final = np.argmax(probs_final, axis=1)


if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_hibrido_svm_stacking_1.csv", index=False)
    print("¡Archivo 'submission_hibrido_svm_stacking_1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")



   Pesos -> SVM: 0.5 | Stacking: 0.5
¡Archivo 'submission_hibrido_svm_stacking_1.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


---
PRUEBAS CON DIFERENTES PESOS

---

In [ ]:
# ESTRATEGIA 1: Favor Stacking (70% Stacking - 30% SVM)
# Lógica: El Stacking tiene 0.57. Le damos el mando. El SVM solo corrige si el Stacking duda mucho.
w_stack = 0.7
w_svm = 0.3
probs_1 = (probs_stacking * w_stack) + (probs_svm * w_svm)
pred_1 = np.argmax(probs_1, axis=1)

# 70/30 empeora resultados. Se abandona la estrategia

# # ESTRATEGIA 2: Dominio Stacking (85% Stacking - 15% SVM)
# # Lógica: Confiamos casi ciegamente en el 0.57, pero dejamos un 15% por si el SVM
# # ve algo geométrico muy obvio que el Stacking se perdió.
# w_stack2 = 0.85
# w_svm2 = 0.15
# probs_2 = (probs_stacking * w_stack2) + (probs_svm * w_svm2)
# pred_2 = np.argmax(probs_2, axis=1)

# # ESTRATEGIA 3: Favor SVM Ligero (60% SVM - 40% Stacking)
# # Lógica: Quizás el 50/50 falló por poco. Probamos darle un empujón al SVM
# # pero manteniendo una base fuerte del Stacking.
# w_stack3 = 0.4
# w_svm3 = 0.6
# probs_3 = (probs_stacking * w_stack3) + (probs_svm * w_svm3)
# pred_3 = np.argmax(probs_3, axis=1)

# --- GENERACIÓN DE ARCHIVOS ---

def save_submission(pred_array, filename):
    if len(pred_array) == len(df_test): # O len(submission)
        # Recuperamos IDs correctos
        ids = df_test.index if 'ID' not in df_test.columns else df_test['ID']

        sub = pd.DataFrame({
            "ID": ids,
            "label": pred_array.astype(int)
        })
        sub.to_csv(filename, index=False)
        print(f"✅ Guardado: {filename}")
    else:
        print(f"❌ Error dimensiones en {filename}")

save_submission(pred_1, "sub_weight_70Stack_30SVM.csv")
save_submission(pred_2, "sub_weight_85Stack_15SVM.csv")
save_submission(pred_3, "sub_weight_40Stack_60SVM.csv")

print("\n🚀 ¡Archivos listos!")

✅ Guardado: sub_weight_70Stack_30SVM.csv
✅ Guardado: sub_weight_85Stack_15SVM.csv
✅ Guardado: sub_weight_40Stack_60SVM.csv

🚀 ¡Archivos listos!


---

ESTRATEGIA RANK BLENDING

---

In [ ]:

def get_rank_probs(probs):
    """Convierte probabilidades en rankings normalizados (0 a 1)"""
    # Aplicamos ranking por columna (por cada clase)
    ranked = np.apply_along_axis(rankdata, 0, probs)
    return ranked / len(probs) # Normalizamos entre 0 y 1

# Pre-calculamos los rankings (Esto pone a ambos modelos en igualdad de condiciones)
# Asumimos que probs_stacking y probs_svm están en memoria
rank_stacking = get_rank_probs(probs_stacking)
rank_svm = get_rank_probs(probs_svm)

# ESTRATEGIA 1: Protección del Líder (90% Stacking - 10% SVM)
# Si el 70/30 falló, reducimos drásticamente la influencia del SVM.
# Usamos probabilidades directas aquí, solo un toque sutil de SVM.
w_stack1 = 0.90
w_svm1 = 0.10
probs_1 = (probs_stacking * w_stack1) + (probs_svm * w_svm1)
pred_1 = np.argmax(probs_1, axis=1)

# ESTRATEGIA 2: Rank Blending (50% - 50%) - RECOMENDADA
# Al usar rankings, eliminamos el "ruido" de la confianza excesiva del SVM.
# Esto suele funcionar mejor cuando los modelos "chocan".
probs_2 = (rank_stacking * 0.5) + (rank_svm * 0.5)
pred_2 = np.argmax(probs_2, axis=1)

# ESTRATEGIA 3: Rank Blending Ponderado (70% Stacking - 30% SVM)
# Lo mismo que tu intento anterior, pero en espacio de Rankings.
# Debería ser mucho más estable que el 0.54 que obtuviste.
probs_3 = (rank_stacking * 0.7) + (rank_svm * 0.3)
pred_3 = np.argmax(probs_3, axis=1)

# --- GENERACIÓN DE ARCHIVOS ---

# def save_submission(pred_array, filename):
#     if len(pred_array) == len(df_test):
#         ids = df_test.index if 'ID' not in df_test.columns else df_test['ID']

#         sub = pd.DataFrame({
#             "ID": ids,
#             "label": pred_array.astype(int)
#         })
#         sub.to_csv(filename, index=False)
#         print(f"Guardado: {filename}")
#     else:
#         print(f"Error dimensiones en {filename}")

# save_submission(pred_1, "sub_RANK_90Stack_10SVM.csv")
# save_submission(pred_2, "sub_RANK_50Stack_50SVM.csv")
# save_submission(pred_3, "sub_RANK_70Stack_30SVM.csv")

# print("\n¡Archivos generados! ")

Guardado: sub_RANK_90Stack_10SVM.csv
Guardado: sub_RANK_50Stack_50SVM.csv
Guardado: sub_RANK_70Stack_30SVM.csv

¡Archivos generados! 


---

CORRECIÓN DEL STACKING EN CASO DE DUDA USANDO EL SVM

---

In [ ]:

# 1. Obtenemos las predicciones base (Clase con mayor probabilidad)
pred_stacking = np.argmax(probs_stacking, axis=1)
pred_svm = np.argmax(probs_svm, axis=1)

# 2. Medimos la "Seguridad" de cada modelo (Probabilidad máxima de la fila)
confianza_stacking = np.max(probs_stacking, axis=1)
confianza_svm = np.max(probs_svm, axis=1)

# # ESTRATEGIA 1: Rescate Conservador (Umbral 0.5 / 0.8)
# # Lógica: Si Stacking duda (<50%) y SVM está muy seguro (>80%), hacemos caso al SVM.
# preds_rescate_1 = pred_stacking.copy()

# # Máscara de corrección
# mask_1 = (confianza_stacking < 0.50) & (confianza_svm > 0.80)
# preds_rescate_1[mask_1] = pred_svm[mask_1]

cambios_1 = np.sum(mask_1)
print(f"ESTRATEGIA 1: Se han corregido {cambios_1} filas donde Stacking dudaba.")

# ESTRATEGIA 2: Rescate Moderado (Umbral 0.6 / 0.7)
# Lógica: Si Stacking no está muy claro (<60%) y SVM tiene una opinión decente (>70%).
preds_rescate_2 = pred_stacking.copy()

mask_2 = (confianza_stacking < 0.60) & (confianza_svm > 0.70)
preds_rescate_2[mask_2] = pred_svm[mask_2]

cambios_2 = np.sum(mask_2)
print(f"ESTRATEGIA 2: Se han corregido {cambios_2} filas (Umbrales más suaves).")

# # ESTRATEGIA 3: El "Juez Supremo" (Max Confidence Wins)
# # Lógica: Simplemente nos quedamos con la predicción del modelo que esté más seguro
# # en esa fila específica. Si SVM tiene 0.9 y Stacking 0.6, gana SVM.
# # Esto evita promedios. Es uno u otro.
# preds_supremo = []
# for i in range(len(probs_stacking)):
#     if confianza_svm[i] > confianza_stacking[i]:
#         preds_supremo.append(pred_svm[i])
#     else:
#         preds_supremo.append(pred_stacking[i])

# preds_supremo = np.array(preds_supremo)
# cambios_3 = np.sum(preds_supremo != pred_stacking)
# print(f"ESTRATEGIA 3: SVM ha ganado la discusión en {cambios_3} filas por mayor confianza.")

ESTRATEGIA 1: Se han corregido 0 filas donde Stacking dudaba.
ESTRATEGIA 2: Se han corregido 2 filas (Umbrales más suaves).
ESTRATEGIA 3: SVM ha ganado la discusión en 29 filas por mayor confianza.


In [ ]:
def save_submission(pred_array, filename):
    if len(pred_array) == len(df_test):
        ids = df_test.index if 'ID' not in df_test.columns else df_test['ID']

        sub = pd.DataFrame({
            "ID": ids,
            "label": pred_array.astype(int)
        })
        sub.to_csv(filename, index=False)
        print(f"Guardado: {filename}")
    else:
        print(f"Error dimensiones en {filename}")

save_submission(preds_rescate_1, "sub_RESCUE_Conservative.csv")
save_submission(preds_rescate_2, "sub_RESCUE_Moderate.csv")
save_submission(preds_supremo, "sub_MAX_CONFIDENCE.csv")

print("\n¡Archivos generados! Estas estrategias NO promedian, solo sustituyen.")

Guardado: sub_RESCUE_Conservative.csv
Guardado: sub_RESCUE_Moderate.csv
Guardado: sub_MAX_CONFIDENCE.csv

¡Archivos generados! Estas estrategias NO promedian, solo sustituyen.


---
## ENSAMBLE JERÁRQUICO

---

In [123]:
print("\n FASE 1: Entrenando SVM Binario")

# Creamos el target binario
# 0 -> 0 (Sano)
# 1,2,3,4 -> 1 (Enfermo)
y_train_binary = (y_train > 0).astype(int)

# Configuración SVM optimizada
svm_binary = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    probability=True,
    random_state=42
)

# Entrenamos con los datos PREP (Pruned)
svm_binary.fit(X_train_prep, y_train_binary)

# Predecimos sobre el Test Prep
# Si predice 0 es Sano. Si predice 1 es "Algo tiene".
preds_binary_svm = svm_binary.predict(X_test_prep)

print(f"   -> SVM ha clasificado como 'Sanos' (0) a {np.sum(preds_binary_svm == 0)} pacientes.")
print(f"   -> SVM ha clasificado como 'Enfermos' (1+) a {np.sum(preds_binary_svm == 1)} pacientes.")


 FASE 1: Entrenando SVM Binario
   -> SVM ha clasificado como 'Sanos' (0) a 80 pacientes.
   -> SVM ha clasificado como 'Enfermos' (1+) a 104 pacientes.


In [124]:
print("\n FASE 2: Entrenando Stacking Multiclase...")

# 2.1 Preparar Datos Completos (Raw + Limpieza Básica + Imputación Simple)
df_clean = limpieza_inicial(df)
X_full_train = df_clean.drop('label', axis=1) if 'label' in df_clean.columns else df_clean
y_full_train = df_clean['label'] if 'label' in df_clean.columns else y_train

df_test_clean_initial = limpieza_inicial(df_test)
X_full_test = df_test_clean_initial.copy()
if 'label' in X_full_test.columns: X_full_test = X_full_test.drop('label', axis=1)

# Limpieza mínima
X_full_train = limpiar_ceros_fisiologicos(X_full_train)
X_full_train = clipear_outliers(X_full_train)
X_full_test = limpiar_ceros_fisiologicos(X_full_test)
X_full_test = clipear_outliers(X_full_test)

# Imputación Simple
imputer = SimpleImputer(strategy='median')
X_full_train = pd.DataFrame(imputer.fit_transform(X_full_train), columns=X_full_train.columns)
X_full_test = pd.DataFrame(imputer.transform(X_full_test), columns=X_full_test.columns)


 FASE 2: Entrenando Stacking Multiclase...


In [125]:
lr_base = LogisticRegression(C=1, penalty='l2', solver='lbfgs', max_iter=1000, random_state=42)
rf_base = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=5, class_weight='balanced', random_state=42)
xgb_base = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, subsample=1.0, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
brf_base = BalancedRandomForestClassifier(n_estimators=200, min_samples_split=2, random_state=42)
eec_base = EasyEnsembleClassifier(n_estimators=10, sampling_strategy='auto', random_state=42)

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', lr_base), ('rf', rf_base), ('xgb', xgb_base), ('brf', brf_base), ('eec', eec_base)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5, n_jobs=-1
)

# Entrenamos con TODOS los datos (0-4) para que aprenda el contexto global
stacking_clf.fit(X_full_train, y_full_train)

# Predicciones Multiclase
preds_multiclass_stacking = stacking_clf.predict(X_full_test)


In [121]:
print("\n FASE 3: Combinando Resultados con estrategia A")

# 1. Empezamos con las predicciones del Stacking (que sabe distinguir 1,2,3,4)
final_predictions = preds_multiclass_stacking.copy()

# 2. APLICAMOS EL FILTRO DEL SVM:
# Donde el SVM dijo "0" (Sano), FORZAMOS que sea 0,
# ignorando lo que diga el Stacking (incluso si el Stacking dijo 1 o 2).
mask_sanos_svm = (preds_binary_svm == 0)
final_predictions[mask_sanos_svm] = 0

#  ¿Qué pasa si SVM dice "Enfermo" pero Stacking dice "0"?
# Opción A: Dejar el 0 del Stacking (Asumimos que SVM detectó algo raro pero Stacking confirmó que no es grave).

# DE MOMENTO: Usamos Opción A (Respetamos el 0 del stacking si pasa el filtro inverso),
# pero la prioridad es limpiar los falsos positivos con el SVM.

print("   Integración completada.")



 FASE 3: Combinando Resultados
   Integración completada.


In [122]:
if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_Hierarchical_SVM_Stacking.csv.csv", index=False)
    print("¡Archivo 'submission_Hierarchical_SVM_Stacking.csv.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

¡Archivo 'submission_Hierarchical_SVM_Stacking.csv.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


In [126]:
print("\n FASE 3: Combinando Resultados (Estrategia B: Corrección Bidireccional)")

# 1. Empezamos con las predicciones del Stacking
final_predictions = preds_multiclass_stacking.copy()

# 2. FILTRO 1: SANAR FALSOS POSITIVOS
# Si SVM dice "0" (Sano), forzamos 0.
mask_sanos_svm = (preds_binary_svm == 0)
num_correciones_a_0 = np.sum((final_predictions != 0) & mask_sanos_svm)
final_predictions[mask_sanos_svm] = 0
print(f"   -> {num_correciones_a_0} pacientes corregidos a SANO (0) por el SVM.")

# 3. FILTRO 2: DETECTAR FALSOS NEGATIVOS (La Opción B)
# Si SVM dice "Enfermo" (1) PERO el Stacking dijo "Sano" (0), forzamos a 1.
mask_enfermos_svm = (preds_binary_svm == 1)
# Solo corregimos si el stacking dijo 0 (si dijo 2, 3 o 4 ya estamos de acuerdo en que está enfermo)
mask_correccion_1 = mask_enfermos_svm & (final_predictions == 0)

num_correciones_a_1 = np.sum(mask_correccion_1)
final_predictions[mask_correccion_1] = 1 # Severidad mínima
print(f"   -> {num_correciones_a_1} pacientes corregidos a ENFERMO LEVE (1) porque el SVM detectó riesgo.")

print("   Integración completada.")


 FASE 3: Combinando Resultados (Estrategia B: Corrección Bidireccional)
   -> 1 pacientes corregidos a SANO (0) por el SVM.
   -> 11 pacientes corregidos a ENFERMO LEVE (1) porque el SVM detectó riesgo.
   Integración completada.


In [127]:
if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_Hierarchical_SVM_Stacking_OptionB.csv", index=False)
    print("¡Archivo 'submission_Hierarchical_SVM_Stacking_OptionB.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

¡Archivo 'submission_Hierarchical_SVM_Stacking_OptionB.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


In [128]:

print("⚡ INICIANDO IMPLANTE QUIRÚRGICO (Base 0.57 + Corrección SVM) ⚡")

# =============================================================================
# 1. CARGAR LA BASE DE ORO (El archivo de 0.57)
# =============================================================================
# Asegúrate de subir este archivo a tu entorno de Colab/Kaggle
file_path_best = "/content/sub_RESCUE_Conservative.csv"

try:
    df_best = pd.read_csv(file_path_best)
    # Aseguramos que tenemos las etiquetas
    preds_base_057 = df_best['label'].values
    print(f"✅ Cargado archivo base (Récord 0.57): {len(preds_base_057)} predicciones.")
except FileNotFoundError:
    raise FileNotFoundError(f"❌ NO SE ENCUENTRA '{file_path_best}'. Súbelo para continuar.")

# =============================================================================
# 2. RE-ENTRENAR SOLO EL SVM (Tu mejor discriminador)
# =============================================================================
# Usamos tu pipeline de preprocesado avanzado (Pruned) que ya tienes en memoria.
# Si X_train_prep / X_test_prep no están definidos, el código fallará.
# Asegúrate de haber ejecutado el bloque de preprocesado antes.

print("\n🛡️ Entrenando SVM Binario (Detector de Riesgo)...")

# Target Binario: 0=Sano, 1=Enfermo (Clases 1,2,3,4)
y_train_binary = (y_train > 0).astype(int)

# Tu mejor configuración SVM (Sin class_weight, como indicaste)
svm_binary = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    class_weight=None,
    probability=True,
    random_state=42
)

svm_binary.fit(X_train_prep, y_train_binary)
preds_binary_svm = svm_binary.predict(X_test_prep)

print("✅ SVM entrenado y predicciones generadas.")


# =============================================================================
# 3. APLICAR LÓGICA JERÁRQUICA SOBRE EL ARCHIVO 0.57
# =============================================================================
print("\n🔗 FUSIONANDO RESULTADOS...")

final_preds = preds_base_057.copy()
cambios_totales = 0

# REGLA 1: SI SVM DICE "SANO" (0) -> FORZAMOS 0
# (Corregimos falsos positivos del Stacking)
mask_force_0 = (preds_binary_svm == 0) & (final_preds != 0)
num_force_0 = np.sum(mask_force_0)
final_preds[mask_force_0] = 0
print(f"   -> {num_force_0} pacientes corregidos a SANO (0).")

# REGLA 2: SI SVM DICE "ENFERMO" (1) Y STACKING DICE "SANO" (0) -> FORZAMOS 1
# (Corregimos falsos negativos del Stacking - Opción B)
mask_force_1 = (preds_binary_svm == 1) & (final_preds == 0)
num_force_1 = np.sum(mask_force_1)
final_preds[mask_force_1] = 1
print(f"   -> {num_force_1} pacientes corregidos a ENFERMO LEVE (1).")

cambios_totales = num_force_0 + num_force_1
print(f"📊 TOTAL CAMBIOS REALIZADOS SOBRE EL 0.57: {cambios_totales}")


# =============================================================================
# 4. GENERAR ARCHIVO FINAL
# =============================================================================
if len(final_preds) == len(df_test):
    ids = df_test.index if 'ID' not in df_test.columns else df_test['ID']

    sub = pd.DataFrame({
        "ID": ids,
        "label": final_preds.astype(int)
    })

    filename = "submission_Surgical_Graft_057_SVM.csv"
    sub.to_csv(filename, index=False)

    print(f"\n🏆 ¡Archivo '{filename}' generado!")
    print("   Este archivo contiene la sabiduría del Stacking (0.57) + la corrección del SVM.")
    print(sub.head())
else:
    print("❌ Error de dimensiones.")

⚡ INICIANDO IMPLANTE QUIRÚRGICO (Base 0.57 + Corrección SVM) ⚡
✅ Cargado archivo base (Récord 0.57): 184 predicciones.

🛡️ Entrenando SVM Binario (Detector de Riesgo)...
✅ SVM entrenado y predicciones generadas.

🔗 FUSIONANDO RESULTADOS...
   -> 1 pacientes corregidos a SANO (0).
   -> 11 pacientes corregidos a ENFERMO LEVE (1).
📊 TOTAL CAMBIOS REALIZADOS SOBRE EL 0.57: 12

🏆 ¡Archivo 'submission_Surgical_Graft_057_SVM.csv' generado!
   Este archivo contiene la sabiduría del Stacking (0.57) + la corrección del SVM.
   ID  label
0   0      2
1   1      0
2   2      0
3   3      3
4   4      0


In [129]:

print("⚡ ENTRENANDO STACKING MEJORADO (Base 0.57 + SVM Integrado) ⚡")

# =============================================================================
# 1. PREPARACIÓN DE DATOS (ESTILO "SIMPLE" DEL 0.57)
# =============================================================================
# Usamos la misma lógica que funcionó en el 0.57: limpieza mínima + imputación simple.

cols_to_int = ['age', 'sex', 'cp', 'restecg']
rename_dict = {
    "age": "edad", "sex": "sexo", "cp": "tipo_dolor_pecho", "trestbps": "tension_en_descanso",
    "chol": "colesterol", "fbs": "azucar", "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto", "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio", "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes", "thal": "estado_corazon_thal"
}

def limpieza_basica(df):
    df = df.copy()
    for col in df.columns:
        if col != 'label':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.rename(columns=rename_dict)
    df.replace([-9, -9.0], np.nan, inplace=True)
    return df

# Cargar datos (Asegúrate de que df y df_test estén cargados en tu entorno)
# Si no, descomenta las líneas de carga:
df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/test.csv')

df_clean = limpieza_basica(df)
df_test_clean = limpieza_basica(df_test)

# Imputación Simple (Mediana) - La clave del éxito del 0.57
imputer = SimpleImputer(strategy='median')

X = df_clean.drop('label', axis=1)
y = df_clean['label']
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Preparar Test
if 'label' in df_test_clean.columns:
    df_test_clean = df_test_clean.drop('label', axis=1)
X_test_imputed = pd.DataFrame(imputer.transform(df_test_clean), columns=df_test_clean.columns)

print("✅ Datos preparados (Estrategia Simple).")

# =============================================================================
# 2. DEFINICIÓN DE MODELOS CON TRATAMIENTO ESPECÍFICO
# =============================================================================
# Aquí está el truco: El SVM necesita Scaler, los Árboles no.
# Usamos Pipelines individuales para cada modelo base.

# A. Modelos de Árboles (Datos "crudos" imputados)
rf = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=5, class_weight='balanced', random_state=42)
xgb = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, subsample=1.0, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
brf = BalancedRandomForestClassifier(n_estimators=200, min_samples_split=2, random_state=42)
eec = EasyEnsembleClassifier(n_estimators=10, sampling_strategy='auto', random_state=42)

# B. Modelos Sensibles a Escala (SVM y LR) -> Llevan StandardScaler incorporado
svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(C=1, gamma=0.05, kernel='rbf', probability=True, random_state=42))
])

lr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=1, penalty='l2', solver='lbfgs', max_iter=1000, random_state=42))
])

# =============================================================================
# 3. CONSTRUCCIÓN DEL STACKING MEJORADO
# =============================================================================
print("🚀 Entrenando Stacking V2 (LR + RF + XGB + BRF + EEC + SVM)...")

stacking_clf_v2 = StackingClassifier(
    estimators=[
        ('lr', lr_pipe),      # Pipeline con Scaler
        ('rf', rf),           # Sin Scaler
        ('xgb', xgb),         # Sin Scaler
        ('brf', brf),         # Sin Scaler
        ('eec', eec),         # Sin Scaler
        ('svm', svm_pipe)     # NUEVO: Pipeline con Scaler e Hiperparámetros Óptimos
    ],
    # Meta-Learner: Usamos LogisticRegression 'balanced' para corregir sesgos finales
    final_estimator=LogisticRegression(max_iter=1000, class_weight='balanced'),
    cv=5,
    n_jobs=-1
)

stacking_clf_v2.fit(X_imputed, y)
print("✅ Entrenamiento completado.")

# =============================================================================
# 4. GENERAR PREDICCIONES
# =============================================================================
print("Generando submission...")
y_pred_final = stacking_clf_v2.predict(X_test_imputed)

# Obtener IDs
submission_ids = df_test.index if 'ID' not in df_test.columns else df_test['ID']

sub = pd.DataFrame({
    "ID": submission_ids,
    "label": y_pred_final.astype(int)
})

filename = "submission_Integrated_Stacking_Plus_SVM.csv"
sub.to_csv(filename, index=False)

print(f"🏆 ¡Archivo '{filename}' guardado!")
print("   Estrategia: Stacking Original Mejorado (SVM añadido correctamente escalado)")
print(sub.head())

⚡ ENTRENANDO STACKING MEJORADO (Base 0.57 + SVM Integrado) ⚡
✅ Datos preparados (Estrategia Simple).
🚀 Entrenando Stacking V2 (LR + RF + XGB + BRF + EEC + SVM)...
✅ Entrenamiento completado.
Generando submission...
🏆 ¡Archivo 'submission_Integrated_Stacking_Plus_SVM.csv' guardado!
   Estrategia: Stacking Original Mejorado (SVM añadido correctamente escalado)
   ID  label
0   0      4
1   1      0
2   2      0
3   3      2
4   4      0


In [130]:

print("⚡ ENTRENANDO STACKING AVANZADO: PIPELINES PARALELOS (SVM Pruned + Árboles Raw) ⚡")

# =============================================================================
# 1. DEFINICIÓN DE CLASES Y FUNCIONES DE PREPROCESADO AVANZADO (Para el SVM)
# =============================================================================

cols_to_int = ['age', 'sex', 'cp', 'restecg']
rename_dict = {
    "age": "edad", "sex": "sexo", "cp": "tipo_dolor_pecho", "trestbps": "tension_en_descanso",
    "chol": "colesterol", "fbs": "azucar", "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto", "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio", "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes", "thal": "estado_corazon_thal"
}
cols_a_clippear = ['tension_en_descanso', 'colesterol', 'latidos_por_minuto', 'cambio_linea_corazon_ejercicio']
categorical_cols_to_round = ['num_venas_grandes', 'estado_corazon_thal', 'sexo', 'tipo_dolor_pecho',
                             'dolor_pecho_con_ejercicio', 'azucar', 'forma_linea_corazon_ejercicio', 'electro_en_descanso']

def limpieza_inicial(df):
    df = df.copy()
    for col in df.columns:
        if col != 'label':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    for col in cols_to_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    df = df.rename(columns=rename_dict)
    df.replace([-9, -9.0], np.nan, inplace=True)
    return df

def limpiar_ceros_fisiologicos(X):
    X = X.copy()
    for col in ['tension_en_descanso', 'colesterol']:
        if col in X.columns:
            X[col] = X[col].replace({0: np.nan, 0.0: np.nan})
    return X

def clipear_outliers(X):
    X = X.copy()
    for col in cols_a_clippear:
        if col in X.columns:
            p1 = X[col].quantile(0.01)
            p99 = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=p1, upper=p99)
    return X

def crear_flags_mnar(df):
    df_new = df.copy()
    cols_mnar = ['num_venas_grandes']
    for col in cols_mnar:
        if col in df_new.columns:
            df_new[f'{col}_is_missing'] = df_new[col].isna().astype(int)
    return df_new

def redondear_imputaciones(X):
    X = X.copy()
    for col in categorical_cols_to_round:
        if col in X.columns:
            X[col] = X[col].round()
    return X

def aplicar_feature_engineering_avanzado(df):
    df = df.copy()
    if 'cambio_linea_corazon_ejercicio' in df.columns:
        df['flag_depresion_st'] = (df['cambio_linea_corazon_ejercicio'] > 0).astype(int)
    if 'tension_en_descanso' in df.columns:
        df['flag_hipertension'] = (df['tension_en_descanso'] > 130).astype(int)
    if 'dolor_pecho_con_ejercicio' in df.columns and 'flag_depresion_st' in df.columns:
        df['score_respuesta_stress'] = df['dolor_pecho_con_ejercicio'] + df['flag_depresion_st']
    cols_comorbilidad = ['azucar', 'flag_hipertension', 'electro_en_descanso']
    if set(cols_comorbilidad).issubset(df.columns):
        electro_punto = (df['electro_en_descanso'] > 0).astype(int)
        df['carga_comorbilidad'] = df['azucar'] + df['flag_hipertension'] + electro_punto
    return df

def ejecutar_pruning_agresivo(df):
    df = df.copy()
    vars_a_eliminar = ['electro_en_descanso', 'colesterol', 'azucar', 'tension_en_descanso']
    cols_existentes = [c for c in vars_a_eliminar if c in df.columns]
    if cols_existentes:
        df = df.drop(columns=cols_existentes)
    return df

class RobustKNNImputerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.scaler = RobustScaler()
        self.imputer = KNNImputer(n_neighbors=n_neighbors, weights='distance')
        self.feature_names_in_ = None
    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns if hasattr(X, 'columns') else [f"feat_{i}" for i in range(X.shape[1])]
        X_scaled = self.scaler.fit_transform(X)
        self.imputer.fit(X_scaled)
        return self
    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        X_imputed_scaled = self.imputer.transform(X_scaled)
        X_imputed = self.scaler.inverse_transform(X_imputed_scaled)
        return pd.DataFrame(X_imputed, columns=self.feature_names_in_, index=X.index)

# =============================================================================
# 2. CONSTRUCCIÓN DE PIPELINES ESPECÍFICOS
# =============================================================================

# PIPELINE A: EL DEL SVM (Complejo + Pruning) - Esto replica tu éxito de 0.565
prep_svm = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear_outliers', FunctionTransformer(clipear_outliers, validate=False)),
    ('mnar_flags', FunctionTransformer(crear_flags_mnar, validate=False)),
    ('imputacion_robusta', RobustKNNImputerWrapper(n_neighbors=5)), # K=5 por defecto
    ('rounding', FunctionTransformer(redondear_imputaciones, validate=False)),
    ('feature_engineering', FunctionTransformer(aplicar_feature_engineering_avanzado, validate=False)),
    ('pruning_agresivo', FunctionTransformer(ejecutar_pruning_agresivo, validate=False)),
    ('final_scaler', RobustScaler())
])

# PIPELINE B: EL DE LOS ÁRBOLES (Simple + Datos Sucios) - Esto replica tu éxito de 0.57
prep_trees = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear', FunctionTransformer(clipear_outliers, validate=False)),
    ('imputer', SimpleImputer(strategy='median'))
    # SIN PRUNING, SIN SCALER, SIN FE COMPLEJO
])

# =============================================================================
# 3. CARGA Y PREPARACIÓN INICIAL
# =============================================================================
# Cargar datos
# df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
# df_test = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/test.csv')

# Limpieza de tipos básica
df_clean = limpieza_inicial(df)
df_test_clean = limpieza_inicial(df_test)

X = df_clean.drop('label', axis=1)
y = df_clean['label']

X_test_final = df_test_clean.copy()
if 'label' in X_test_final.columns:
    X_test_final = X_test_final.drop('label', axis=1)

print("✅ Datos base cargados. Iniciando entrenamiento de Stacking Paralelo...")

# =============================================================================
# 4. DEFINICIÓN DEL STACKING CON PIPELINES INTEGRADOS
# =============================================================================

# Envolvemos cada modelo con SU pipeline de preprocesado correspondiente

# Grupo Árboles (Usan prep_trees)
rf_pipe = Pipeline([('prep', prep_trees), ('rf', RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=5, class_weight='balanced', random_state=42))])
xgb_pipe = Pipeline([('prep', prep_trees), ('xgb', XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, subsample=1.0, use_label_encoder=False, eval_metric='mlogloss', random_state=42))])
brf_pipe = Pipeline([('prep', prep_trees), ('brf', BalancedRandomForestClassifier(n_estimators=200, min_samples_split=2, random_state=42))])
eec_pipe = Pipeline([('prep', prep_trees), ('eec', EasyEnsembleClassifier(n_estimators=10, sampling_strategy='auto', random_state=42))])

# Grupo SVM (Usa prep_svm) - AQUÍ ESTÁ LA MAGIA
svm_pipe_full = Pipeline([
    ('prep', prep_svm),
    ('svm', SVC(C=1, gamma=0.05, kernel='rbf', probability=True, random_state=42))
])

# Grupo LR (Usa prep_svm también, le viene bien el escalado y limpieza)
lr_pipe_full = Pipeline([
    ('prep', prep_svm),
    ('lr', LogisticRegression(C=1, penalty='l2', solver='lbfgs', max_iter=1000, random_state=42))
])

print("🚀 Entrenando Stacking Final...")

stacking_clf_final = StackingClassifier(
    estimators=[
        ('lr', lr_pipe_full),   # Recibe datos limpios
        ('rf', rf_pipe),        # Recibe datos sucios
        ('xgb', xgb_pipe),      # Recibe datos sucios
        ('brf', brf_pipe),      # Recibe datos sucios
        ('eec', eec_pipe),      # Recibe datos sucios
        ('svm', svm_pipe_full)  # Recibe datos limpios (PRUNED)
    ],
    final_estimator=LogisticRegression(max_iter=1000, class_weight='balanced'),
    cv=5,
    n_jobs=-1
)

stacking_clf_final.fit(X, y)
print("✅ Entrenamiento completado.")

# =============================================================================
# 5. GENERAR SUBMISSION
# =============================================================================
print("Generando submission...")
y_pred_final = stacking_clf_final.predict(X_test_final)

submission_ids = df_test.index if 'ID' not in df_test.columns else df_test['ID']
sub = pd.DataFrame({
    "ID": submission_ids,
    "label": y_pred_final.astype(int)
})

filename = "submission_Stacking_Parallel_Pipelines.csv"
sub.to_csv(filename, index=False)

print(f"🏆 ¡Archivo '{filename}' guardado!")
print("   Estrategia: Cada modelo recibe sus datos ideales (SVM->Pruned, Árboles->Raw)")
print(sub.head())

⚡ ENTRENANDO STACKING AVANZADO: PIPELINES PARALELOS (SVM Pruned + Árboles Raw) ⚡
✅ Datos base cargados. Iniciando entrenamiento de Stacking Paralelo...
🚀 Entrenando Stacking Final...
✅ Entrenamiento completado.
Generando submission...
🏆 ¡Archivo 'submission_Stacking_Parallel_Pipelines.csv' guardado!
   Estrategia: Cada modelo recibe sus datos ideales (SVM->Pruned, Árboles->Raw)
   ID  label
0   0      4
1   1      0
2   2      0
3   3      4
4   4      0


---

## DOCUMENTACIÓN

---

El objetivo de esta fase fue intentar batir el récord interno del equipo (**0.57**, logrado por un *Stacking Random Classifier* en una línea paralela) mediante el uso de arquitecturas avanzadas y un preprocesado quirúrgico.

### 1. 🤖 Estrategia de Modelado: El Foco en Árboles
La mayoría de los ensambles de esta iteración se construyeron priorizando modelos de **árboles no lineales** (Random Forest, XGBoost, CatBoost).
* **Racional:** Se eligió este enfoque porque fue precisamente esta familia de algoritmos la que logró el récord de **0.57** utilizando un preprocesado simple. Se intentó replicar ese éxito combinándolos con nuevas técnicas de limpieza.

### 2. 🧪 Resultados y Eficiencia (El Retorno del SVM)
A pesar de la complejidad de los ensambles probados, el análisis de resultados revela un hallazgo sobre la eficiencia:
* **Mejor Retorno por Complejidad:** El **SVM Individual (Optimizado + Pruned)** logró un score de **0.565**, empatando técnicamente con los ensambles jerárquicos más complejos de esta fase.
* **Techo de la Iteración:** Ninguna de las arquitecturas propuestas (Stacking Híbrido, Pipelines Paralelos) logró superar el **0.57** del *Stacking Random Classifier* de referencia. La complejidad añadida en esta fase no aportó valor incremental sobre el modelo base bien preprocesado.

### 3. 📉 Conclusión Técnica
Se ha demostrado que, bajo el esquema de preprocesado avanzado ("Pruning"), un modelo geométrico simple (SVM) es capaz de igualar a ensambles masivos. Sin embargo, para romper la barrera del 0.57, la sofisticación del modelo no es suficiente; el límite parece estar en la estructura actual de los datos.

---

### 📝 P.D. Análisis Competitivo (El salto al 0.60)
Se observa que equipos rivales han alcanzado puntuaciones de **0.59 - 0.60**.
* **Diagnóstico:** Dado que nuestros modelos (tanto lineales como no lineales) se han estancado en el rango 0.54-0.57 independientemente de la arquitectura, es altamente probable que la ventaja de los rivales no provenga del modelo, sino de la **información**.
* **Hipótesis:** Es posible que hayan logrado desbloquear la "Clase 2" (el punto ciego de nuestros modelos) mediante:
    1.  **Ingeniería de Características de Dominio:** Creación de variables no lineales muy específicas que nosotros hemos eliminado en el *pruning*.
    3.  **Preprocesado Diferencial:** Una imputación que preserve mejor la varianza de los grados intermedios de enfermedad que nuestro enfoque robusto.